<a href="https://colab.research.google.com/github/ram130849/Deep_Learning_Systems_Assignments/blob/main/TensorFlow/Tejas/Deep_Learning_HW3_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Homework 3

References:

Garbage collector: https://stackoverflow.com/questions/1316767/how-can-i-explicitly-free-memory-in-python

Checkpoint code and concept from : https://keras.io/api/callbacks/model_checkpoint/

In [1]:
#Path: TensorFlow/Tejas/
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.9.2


In [3]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
2.151927360000002
GPU (s):
0.0389262429999917
GPU speedup over CPU: 55x


##Dependencies

In [4]:
!pip install tensorflow-addons


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 4.3 MB/s 


In [5]:
import tensorflow_addons as tfa

In [6]:
import numpy as np
import random
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Model
import keras.utils
from keras import utils as np_utils
from keras.models import Sequential
from keras.layers import  Dense, Dropout, Activation, Flatten
from tensorflow.keras import optimizers
from librosa.core import stft,istft
import soundfile as sf
from sklearn.model_selection import train_test_split

import tensorflow as tf
%matplotlib inline

##Load Data Set

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
cifar10_data = tf.keras.datasets.cifar10
(x_train, Y_train), (x_test, Y_test) = cifar10_data.load_data()

170498071/170498071 [==============================] - 11s 0us/step


In [9]:
x_train.shape

(50000, 32, 32, 3)

##Rescale to [0,1]

In [10]:
x_train = x_train/255
x_test = x_test/255

###Labelled and Unlabelled

In [11]:
X_train_label,X_train_unlabelled = x_train[49500:50000],x_train[0:49500]


In [12]:
y_train_label,y_train_unlabelled = Y_train[49500:50000],Y_train[0:49500]

##PreText Task

###Class 0 
Ref: https://www.tensorflow.org/api_docs/python/tf/zeros

In [13]:
X_train0 = tf.convert_to_tensor(X_train_unlabelled)

In [14]:
y_train0 = tf.zeros(shape = y_train_unlabelled.shape,dtype=tf.int32)

In [15]:
y_train0.shape # Just labels hence its (49500,1)

TensorShape([49500, 1])

In [16]:
print("shape of X : {} and Y is {}".format(X_train0.shape,y_train0.shape))

shape of X : (49500, 32, 32, 3) and Y is (49500, 1)


###Class 1 
Ref: 

https://www.tensorflow.org/api_docs/python/tf/image/flip_up_down

https://www.tensorflow.org/api_docs/python/tf/ones

In [17]:
X_train1 = tf.image.flip_up_down(X_train_unlabelled)

In [18]:
y_train1 = tf.ones(shape=y_train_unlabelled[0:49500].shape,dtype=tf.int32)

In [19]:
print("shape of X : {} and Y is {}".format(X_train1.shape,y_train1.shape))

shape of X : (49500, 32, 32, 3) and Y is (49500, 1)


In [20]:
y_train1

<tf.Tensor: shape=(49500, 1), dtype=int32, numpy=
array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]], dtype=int32)>

###Class 2

In [21]:
X_train2 = tf.image.rot90(X_train_unlabelled)

In [22]:
y_train2 = 2*tf.ones(shape=y_train_unlabelled[0:49500].shape,dtype=tf.int32)

In [23]:
print("shape of X : {} and Y is {}".format(X_train2.shape,y_train2.shape))


shape of X : (49500, 32, 32, 3) and Y is (49500, 1)


##Rescale Data to [-1,1]

In [24]:
def rescale(data):
  data = tf.math.divide(
   tf.subtract(
      data, 
      tf.reduce_min(data)
   ), 
   tf.subtract(
      tf.reduce_max(data), 
      tf.reduce_min(data)
   )
  )
  return 2*data-1

In [25]:
X_train_label = X_train_label - X_train_label.mean()

In [26]:
X_train_unlabelled = X_train_unlabelled - X_train_unlabelled.mean()

In [27]:
x_test = x_test - x_test.mean()

In [28]:
re_X_train0 = tf.cast(X_train0,tf.float32)
re_X_train1 = tf.cast(X_train1,tf.float32)
re_X_train2 = tf.cast(X_train2,tf.float32)


###Concat

In [32]:
X_train_merged = tf.concat([X_train0, X_train1,X_train2], 0)

InvalidArgumentError: ignored

In [ ]:
y_train_merged = tf.concat([y_train0, y_train1,y_train2], 0)

###Pre Text Model



In [ ]:
pretextmodel = keras.Sequential([
    keras.layers.Conv2D(filters = 10,kernel_size = (5,5),strides = 1, activation = 'relu',data_format='channels_last',input_shape=(32,32,3)),
    keras.layers.MaxPool2D(pool_size = (2,2),strides = 2),
    keras.layers.Conv2D(filters = 10,kernel_size = (5,5),strides = 1, activation = 'relu',data_format='channels_last' ),
    keras.layers.MaxPool2D(pool_size = (2,2),strides = 2),
    keras.layers.Flatten(),
    keras.layers.Dense(units = 20, activation = 'relu', kernel_initializer='he_normal'),
    keras.layers.Dense(units = 3, activation = 'softmax', kernel_initializer = 'he_normal')
])

In [ ]:
pretextmodel.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
pretextmodel.summary()

Checkpoint code and concept from : https://keras.io/api/callbacks/model_checkpoint/

In [ ]:
history = pretextmodel.fit(X_train_merged, y_train_merged, batch_size=128, epochs=100)

In [ ]:
pretextmodel.save('/content/drive/MyDrive/ENGR-E-533 /DATA/HW3/Models/pretext_model.h5')

##Baseline Model

In [ ]:
baselinemodel = keras.Sequential([
    keras.layers.Conv2D(filters = 10,kernel_size = (5,5),strides = 1, activation = 'relu',data_format='channels_last',input_shape=(32,32,3)),
    keras.layers.MaxPool2D(pool_size = (2,2),strides = 2),
    keras.layers.Conv2D(filters = 10,kernel_size = (5,5),strides = 1, activation = 'relu',data_format='channels_last' ),
    keras.layers.MaxPool2D(pool_size = (2,2),strides = 2),
    keras.layers.Flatten(),
    keras.layers.Dense(units = 20, activation = 'relu', kernel_initializer='he_normal'),
    keras.layers.Dense(units = 10, activation = 'softmax', kernel_initializer = 'he_normal')
])

In [ ]:
baselinemodel.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])


In [ ]:
baselinemodel.summary()

In [ ]:
baselinehistory = baselinemodel.fit(X_train_label, 
          y_train_label,  
          epochs=10000,
          validation_data = (x_test, Y_test),
          verbose=1,
          batch_size=256)

In [ ]:
baselinemodel.save('/content/drive/MyDrive/ENGR-E-533 /DATA/HW3/Models/baseline_model.h5')

##Transfer Learning

Ref and code: https://www.tensorflow.org/js/guide/models_and_layers

Layer wise optimizer customization: https://www.tensorflow.org/addons/api_docs/python/tfa/optimizers/MultiOptimizer

In [ ]:
TL_model = tf.keras.models.load_model('/content/drive/MyDrive/ENGR-E-533 /DATA/HW3/Models/pretext_model.h5')

In [ ]:
TL_model.get_weights()

In [ ]:
TL_model.summary()

In [ ]:
TL_model.pop()

In [ ]:
TL_model.add(Dense(10, activation='softmax', kernel_initializer='he_normal'))

In [ ]:
TL_model.summary()


In [ ]:
optimizers = [
    tf.keras.optimizers.Adam(learning_rate=1e-6),
    tf.keras.optimizers.Adam(learning_rate=1e-3)
]

In [ ]:
optimizers_and_layers = [(optimizers[0], TL_model.layers[:-1]), (optimizers[1], TL_model.layers[-1])]

In [ ]:
optimizer1 = tfa.optimizers.MultiOptimizer(optimizers_and_layers)

In [ ]:
TL_model.compile(optimizer=optimizer1,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])


In [ ]:
TL_history = TL_model.fit(X_train_label, 
          y_train_label,  
          epochs=10000,
          validation_data = (x_test, Y_test),
          verbose=1,
          batch_size=512)